In [1]:
from langchain.graphs import Neo4jGraph
import os
import glob
import json
from pathlib import Path
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts.prompt import PromptTemplate

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

/Users/nadcharin/.local/share/virtualenvs/harry-potter-demo-M6hG0Rsr/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:

child_store = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(),
    url=os.getenv('NEO4J_URL'),
    username=os.getenv('NEO4J_USER'),
    password=os.getenv('NEO4J_PASS'),
    database=os.getenv('NEO4J_DB'),
    index_name='child_document',
    )

vector_child_qa = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(temperature=0,model_name=os.getenv('OPENAI_MODEL')), chain_type="stuff", retriever=child_store.as_retriever()
    )


CYPHER_GENERATION_TEMPLATE = """You are an expert Neo4j Cypher translator who understands the question in english and convert to Cypher strictly based on the Neo4j Schema provided and following the instructions below:
    1. Generate Cypher query compatible ONLY for Neo4j Version 5
    2. DO NOT use EXISTS, SIZE keywords in the cypher. Use alias when using the WITH keyword
    3. Use only Nodes and relationships mentioned in the schema
    4. Always enclose the Cypher output inside 3 backticks
    5. ALWAYS do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Patient name use `toLower(p.name) contains 'neo4j'`
    6. Always use aliases to refer the node in the query
    7. Cypher is NOT SQL. So, do not mix and match the syntaxes
    Schema:
    {schema}

    Note: Do not include any explanations or apologies in your responses.
    Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
    Do not include any text except the generated Cypher statement.

    The question is:
    {question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
        input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE)


graph = Neo4jGraph(url=os.getenv('NEO4J_URI'), username=os.getenv('NEO4J_USER'), password=os.getenv('NEO4J_PASSWORD'),database=os.getenv('NEO4J_DATABASE'))

cypher_chain = GraphCypherQAChain.from_llm(
cypher_llm = ChatOpenAI(temperature=0, model_name=os.getenv('OPENAI_MODEL')),
qa_llm = ChatOpenAI(temperature=0,model_name=os.getenv('OPENAI_MODEL')), graph=graph, verbose=True,cypher_prompt=CYPHER_GENERATION_PROMPT)


In [3]:
tools = [
    Tool(
        name="Vector",
        func=vector_child_qa.run,
        description="""Useful when you need to answer general questions or summarisation.
        Not useful for counting the number of anything.
        Use full question as input.
        """,
    ),
    Tool(
        name="Graph",
        func=cypher_chain.run,
        description="""Useful when you need to answer questions about relationship,
        their dependencies. Also useful for any sort of
        aggregation like counting the number, etc.
        Use full question as input.
        """,
    ),
]

mrkl = initialize_agent(
    tools, ChatOpenAI(temperature=0, model_name=os.getenv('OPENAI_MODEL')), agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

In [4]:
response = mrkl.run("Which actors appeared in all eight Harry Potter films, and who are their characters?")
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `Which actors appeared in all eight Harry Potter films, and who are their characters?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (actor:Cast)-[acted:ACTED_IN]->(movie:Movie)
WHERE toLower(movie.title) CONTAINS 'harry potter'
WITH actor, acted.role AS character, COUNT(movie) AS movieCount
WHERE movieCount = 8
RETURN actor.name, COLLECT(character) AS characters

Full Context:
[{'actor.name': 'Tom Felton', 'characters': ['Draco Malfoy']}, {'actor.name': 'Bonnie Wright', 'characters': ['Ginny Weasley']}, {'actor.name': 'Emma Watson', 'characters': ['Hermione Granger']}, {'actor.name': 'Rupert Grint', 'characters': ['Ron Weasley']}, {'actor.name': 'Daniel Radcliffe', 'characters': ['Harry Potter']}]

> Finished chain.
Tom Felton appeared in all eight Harry Potter films as Draco Malfoy, Bonnie Wright as Ginny Weasley, Emma Watson as Hermione Granger, Rupert Grint as Ron Weasley, and Daniel Radcliffe

In [5]:
response = mrkl.run("How does Harry Potter discover he is a wizard?")
print(response)



> Entering new AgentExecutor chain...

Invoking: `Vector` with `How does Harry Potter discover he is a wizard?`


Harry Potter discovers he is a wizard just before his eleventh birthday when owls begin delivering letters addressed to him from Hogwarts School of Witchcraft and Wizardry. Despite his aunt and uncle's (the Dursleys) efforts to keep this information from him by refusing to let him read the letters and even fleeing to an isolated island hut, Hagrid, the Keeper of Keys and Grounds at Hogwarts, arrives in person to deliver Harry's acceptance letter. Hagrid also informs Harry about his magical heritage, including the fact that his parents, James and Lily Potter, were a wizard and a witch who were killed by the dark wizard Lord Voldemort.Harry Potter discovers he is a wizard just before his eleventh birthday when Rubeus Hagrid, the Keeper of Keys and Grounds at Hogwarts, arrives to personally deliver Harry's acceptance letter to Hogwarts School of Witchcraft and Wizardry. Prio